# Bitcoin hash functions

Here we'll cover four distinct functions that will be used throughout: SHA256 (OP_SHA256), RIPEMD160 (OP_RIPEMD160), HASH256 (OP_HASH256), and HASH160 (OP_HASH160). We'll use the python library `hashlib` to import functions for SHA256 and RIPEMD160, then with those we can create our own functions for HASH256 and HASH160.

## SHA256 (OP_SHA256)
SHA256 (Secure Hashing Algorithm 256) is a hash function that takes in an array of bytes of any length and outputs a 256 bit (32 byte) hash. To do this in python we'll use `sha256` method from the `hashlib` library.

In [1]:
import hashlib

data = bytes.fromhex("deadbeef")
output = hashlib.sha256(data).digest()
print(output.hex())

5f78c33274e43fa9de5659265c1d917e25c03722dcb0b8d27db8d5feaa813953


## RIPEMD160 (OP_RIPEMD160)
RIPEMD160 (RIPE Message Digest) is a hash function that takes an array of bytes of any length and outputs a 160bit (20 byte) hash. The `hashlib` library has a function for RIPEMD160 we can use.

In [2]:
output = hashlib.new('ripemd160', data).digest()
print(output.hex())

226821c2f5423e11fe9af68bd285c249db2e4b5a


## HASH256 (OP_HASH256)
HASH256 is simply two rounds of SHA256. The output of the first round is put directly into a second round of hashing. It is used as the hashing function behind proof of work mining, but it is also used in various places for creating transactions.

Note that in many places bitcoind displays the output of SHA256 in little endian notation, such as the transaction id (txid) and block hash. In python we can reverse the order of bytes using the shorthand `[::-1]`. An example of doing such a is given below in `Example usage`, where we use it to display the txid in little endian.

In [3]:
import hashlib
def hash256(data: bytes):
    '''Two rounds of SHA256 (aka Hash256)'''
    hash_1 = hashlib.sha256(data).digest()
    hash_2 = hashlib.sha256(hash_1).digest()
    return hash_2

### Example HASH256 (block hash)
In this example we'll use the `hash256` function to calculate the block hash of [block 125552](https://mempool.space/block/00000000000000001e8d6829a8a21adc5d38d0a473b144b6765798e61f98bd1d). Note that bitcoind displays the output of `hash256` in little endian whereas the output of SHA-256 is displayed as big endian in our function.

In [4]:
# The block header from block 125552
header_hex = (
    "01000000" +
     "81cd02ab7e569e8bcd9317e2fe99f2de44d49ab2b8851ba4a308000000000000" +
     "e320b6c2fffc8d750423db8b1eb942ae710e951ed797f7affc8892b0f1fc122b" +
     "c7f5d74d" +
     "f2b9441a" +
     "42a14695")

# Convert the hex string to a byte array
header_bytes = bytes.fromhex(header_hex)

# Use our hash256 function to compute the block hash
blockhash = hash256(header_bytes)

# To convert the block hash to little endian we use the shorthand `[::-1]` to reverse the byte order
blockhash_le = blockhash[::-1]

# use .hex() to convert the byte array into a human readable hex
print(blockhash_le.hex())

00000000000000001e8d6829a8a21adc5d38d0a473b144b6765798e61f98bd1d


## HASH160 (OP_HASH160)
The second hashing function we'll use is HASH160. This is one round of SHA256 followed by one round of RIPEMD160. Unlike SHA256, the output of RIPEMD160 is a 160 bit (20 byte) hash. HASH160 has an op code `OP_HASH160`, and is used in the script pubkeys for p2pkh and p2wpkh transactions. It is also as a hashlock mechanism for second layer protocols such as in lightning HTLCs or cross chain swaps. 

In [5]:
def hash160(data: bytes):
    '''sha256 followed by ripemd160'''
    hash_1 = hashlib.sha256(data).digest()
    hash_2 = hashlib.new('ripemd160', hash_1).digest()
    return hash_2

### Example HASH160 (pubkey hash)
Here we'll calculate the hash for a compressed pubkey using our hash160 function:

In [6]:
pubkey = bytes.fromhex("11")
pkh = hash160(pubkey)
print(pkh.hex())

ca61e52e881d41374e640f819cd118cc153b21a7


## Quiz
- Q1: The txid is calculated by performing two rounds of SHA-256 on the full tx hex (ignoring any witness fields). Given this tx hex for a p2pkh transaction. Which of the following is the txid? Note that bitcoind displays the txid in little endian notation.
    tx hex: `0100000001399434f3943d776250cf4b4c2b3fa2cac259dd5551a822e1976d25a2d9e0231d010000006b483045022100a628f785b81d
04e3b5d2f4a554c839acab64215935a0558dda1e33d0120dada30220616acf4b1c796cebe11f30dd53bbb4354899d924a4791c8fd7c0ae3
da0c4782c0121034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aaffffffff0280d1f008000000001976a9
143bc28d6d92d9073fb5e3adf481795eaf446bceed88ac80f0fa02000000001976a914531260aa2a199e228c537dfa42c82bea2c7c1f4d8
8ac00000000`
   - A. `6705c06db11e0752d0ffb469c97952995b7ad25d0d2dfca770faddc85dc0e40b`
   - B. `f6ed6c8ca58680cc5102ee6950accc9aef9c0c50d823db238aebc2f7af2e08d7`
   - C. `ceb38b3a4bd6acb3f1a2ddcc8526bffe8a1d03a64d3ebed95b11cf4eb4e63bdd`
   - D. `596711e622bf8dc1d6b892863d56504003f39bb0ee482d6d1715e590e58b35bc`
    

- Q2: Given the public key `034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aa`, which of the following is the public key hash?
    - A. `fc7250a211deddc70ee5a2738de5f07817351cef`
    - B. `aeb3fad5678d4902af497a832348593c87978689`
    - C. `4b3208850947db1fd9ad3200092bf49ce7a1bef2`
    - D. `cb2ed68e9d1c89694579736b4800d4aaf5ec7546`

## Answers
A1: D - `596711e622bf8dc1d6b892863d56504003f39bb0ee482d6d1715e590e58b35bc`

In [7]:
tx_hex = "0100000001399434f3943d776250cf4b4c2b3fa2cac259dd5551a822e1976d25a2d9e0231d010000006b483045022100a628f785b81d04e3b5d2f4a554c839acab64215935a0558dda1e33d0120dada30220616acf4b1c796cebe11f30dd53bbb4354899d924a4791c8fd7c0ae3da0c4782c0121034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aaffffffff0280d1f008000000001976a9143bc28d6d92d9073fb5e3adf481795eaf446bceed88ac80f0fa02000000001976a914531260aa2a199e228c537dfa42c82bea2c7c1f4d88ac00000000"
tx = bytes.fromhex(tx_hex)
txid = hash256(tx)
txid[::-1].hex()

'596711e622bf8dc1d6b892863d56504003f39bb0ee482d6d1715e590e58b35bc'

A2: A - `fc7250a211deddc70ee5a2738de5f07817351cef`

In [8]:
pk = bytes.fromhex("034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aa")
hash160(pk).hex()

'fc7250a211deddc70ee5a2738de5f07817351cef'